In [ ]:
import os
import torch
import numpy as np
from glob import glob

def extract_feature_tensor(sample):
    """Extrahiert das Feature-Matrix-Tensor aus einem Sample (tuple, dict oder direkt)."""
    if isinstance(sample, tuple):
        x = sample[0]
    elif isinstance(sample, dict):
        x = sample.get("features", None)
        if x is None:
            raise ValueError("Kein 'features'-Key im Dictionary-Sample.")
    else:
        x = sample

    if isinstance(x, np.ndarray):
        x = torch.tensor(x, dtype=torch.float32)
    elif not torch.is_tensor(x):
        raise TypeError(f"Unbekannter Datentyp: {type(x)}")
    return x

# === Verzeichnisse ===
input_root = "montage_datasets/"
output_root_spectral = "montage_datasets/spectral_only_small/"
output_root_temporal = "montage_datasets/temporal_only_small/"

os.makedirs(output_root_spectral, exist_ok=True)
os.makedirs(output_root_temporal, exist_ok=True)

configs = [f for f in os.listdir(input_root) if os.path.isdir(os.path.join(input_root, f))]
print(len(configs))
for config in configs:
    input_dir = os.path.join(input_root, config)
    out_spec = os.path.join(output_root_spectral, config)
    out_temp = os.path.join(output_root_temporal, config)
    os.makedirs(out_spec, exist_ok=True)
    os.makedirs(out_temp, exist_ok=True)

    pt_files = sorted(glob(os.path.join(input_dir, "*.pt")))

    for file_path in pt_files:
        try:
            samples = torch.load(file_path)
            # Falls es sich nicht um eine Liste handelt, mache eine Liste daraus
            if not isinstance(samples, list):
                samples = [samples]

            spectral_samples = []
            temporal_samples = []

            for sample in samples:
                # Entpacke Sample
                if isinstance(sample, tuple):
                    feature_matrix, label, eeg_id, *rest = sample
                elif isinstance(sample, dict):
                    feature_matrix = sample.get("features")
                    label = sample.get("label")
                    eeg_id = sample.get("eeg_id")
                    rest = []
                else:
                    raise ValueError("Unbekanntes Sample-Format")

                # In Tensor umwandeln (falls nötig)
                if isinstance(feature_matrix, np.ndarray):
                    feature_matrix = torch.tensor(feature_matrix, dtype=torch.float32)

                # Split mit alter Feature-Reihenfolge
                spectral = torch.cat([feature_matrix[:, :10], feature_matrix[:, 13:14]], dim=1)
                temporal = torch.cat([feature_matrix[:, 10:13], feature_matrix[:, 14:15]], dim=1)

                # Originalstruktur beibehalten
                spectral_samples.append((spectral, label, eeg_id, *rest))
                temporal_samples.append((temporal, label, eeg_id, *rest))

            # Speichern
            base_name = os.path.basename(file_path)
            torch.save(spectral_samples, os.path.join(out_spec, base_name))
            torch.save(temporal_samples, os.path.join(out_temp, base_name))

            #print(f"{base_name} in {config} erfolgreich aufgeteilt.")

        except Exception as e:
            print(f"Fehler bei {file_path}: {e}")

18


In [ ]:
import torch
import os

path = "montage_datasets/spectral_only/win4_step1/"  # or temporal_only/...

# Load one sample
file = os.listdir(path)[0]
data = torch.load(os.path.join(path, file))

print(f"Loaded {file}, contains {len(data)} samples")

# Inspect the first sample
sample = data[0]
features = sample[0]
label = sample[1]

# Show types and shapes
print("Feature shape:", features.shape)
print("Label:", label)
print("Feature type:", type(features))
print("Label type:", type(label))

In [ ]:
data_folder = "montage_datasets/win4_step1/"
file_paths = sorted(glob(os.path.join(data_folder, "*.pt")))
print(len(file_paths))

In [ ]:
import os
import torch
import numpy as np
from glob import glob

def extract_feature_tensor(sample):
    """Extrahiert das Feature-Matrix-Tensor aus einem Sample (tuple, dict oder direkt)."""
    if isinstance(sample, tuple):
        x = sample[0]
    elif isinstance(sample, dict):
        x = sample.get("features", None)
        if x is None:
            raise ValueError("Kein 'features'-Key im Dictionary-Sample.")
    else:
        x = sample

    if isinstance(x, np.ndarray):
        x = torch.tensor(x, dtype=torch.float32)
    elif not torch.is_tensor(x):
        raise TypeError(f"Unbekannter Datentyp: {type(x)}")
    return x

# === Verzeichnisse ===
input_root = "montage_datasets/win4_step1"
output_root_spectral = "montage_datasets/spectral_only_small/"
output_root_temporal = "montage_datasets/temporal_only_small/"

os.makedirs(output_root_spectral, exist_ok=True)
os.makedirs(output_root_temporal, exist_ok=True)



pt_files = sorted(glob(os.path.join(input_dir, "*.pt")))

for file_path in pt_files:
try:
    samples = torch.load(file_path)
    spectral_samples = []
    temporal_samples = []

    for sample in samples:
        # Entpacke Sample
        if isinstance(sample, tuple):
            feature_matrix, label, eeg_id, *rest = sample
        elif isinstance(sample, dict):
            feature_matrix = sample.get("features")
            label = sample.get("label")
            eeg_id = sample.get("eeg_id")
            rest = []
        else:
            raise ValueError("Unbekanntes Sample-Format")

        # In Tensor umwandeln (falls nötig)
        if isinstance(feature_matrix, np.ndarray):
            feature_matrix = torch.tensor(feature_matrix, dtype=torch.float32)

        # Split mit alter Feature-Reihenfolge
        spectral = torch.cat([feature_matrix[:, :10], feature_matrix[:, 13:14]], dim=1)
        temporal = torch.cat([feature_matrix[:, 10:13], feature_matrix[:, 14:15]], dim=1)

        # Originalstruktur beibehalten
        spectral_samples.append((spectral, label, eeg_id, *rest))
        temporal_samples.append((temporal, label, eeg_id, *rest))

    # Speichern
    base_name = os.path.basename(file_path)
    torch.save(spectral_samples, os.path.join(out_spec, base_name))
    torch.save(temporal_samples, os.path.join(out_temp, base_name))

    #print(f"{base_name} in {config} erfolgreich aufgeteilt.")

except Exception as e:
    print(f"Fehler bei {file_path}: {e}")